In [ ]:
import lyricsgenius as genius
import time
import os

### Lastfm template

In [ ]:
def lastfm_get(payload):
    # define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Add API key and format to the payload
    payload['api_key'] = API_KEY
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response

def lookup_tags_for_artist_song(row):
    artist, song = row["Artist"], row["SName"]
    
    response = lastfm_get({
        'method': 'track.getTopTags',
        'artist':  artist,
        'track': song
    })

    # if there's an error, just return nothing
    if response.status_code != 200 or "error" in response.text:
        return "NoSongFound"

    # extract the top tags and turn them into a string
    tags = [t['name'] for t in response.json()['toptags']['tag']]
    tags_str = ', '.join(tags)
    if tags_str == "":
        tags_str ="NoTagsFound" 
    # TODO: Could later also look at track tag count

    # rate limiting
    if not getattr(response, 'from_cache', False):
        # TODO: Can I reduce the time?
        time.sleep(0.25)
    return tags_str

tqdm.pandas()

artists_songs_dataframe['Tags'] = artists_songs_dataframe[["Artist", "SName"]].progress_apply(lambda row: lookup_tags_for_artist_song(row), axis=1)

# Save df
artists_songs_dataframe.to_csv('data/tags_first_third.csv', index=False)

## Download lyrics from Genius

In [ ]:
LYRICS_DIR = os.path("../data/genius_lyrics")

In [ ]:
# TODO: Add this function to python module file
def scrape_lyrics(song_name, artist_name):
    # TODO: Change the docstring depending on decision below
    """Scrape lyrcis on Genius and returns the lyrics.

    :param song_name: song name of the lyrics we want to scrape.
    :param artist_name: artist name of the lyrics we want to scrape.
    
    :return: Lyrics of given song and artist name.
    :rtype: String or None
    """

    api_token = "TQdTzth8QuovVPC3AWrU4EPADP2v2mil2_TXw0xWA-50wmFgsK8-04UWD6vRRLkR"

    api = genius.Genius(client_access_token=api_token, verbose=False)

    # Search for song
    song = api.search_song(song_name, artist_name)

    if song is not None:
        # save_lyrics function: https://github.com/johnwmillr/LyricsGenius/blob/master/lyricsgenius/song.py
        # TODO: Compare json and txt format
        print("Song found.")
        # TODO: Make saving lyrics optional? Add bool parameter and format parameter? 
        song.save_lyrics(filename=f"{LYRICS_DIR}/{song_name}_{artist_name}.json",
                            overwrite=True, verbose=False, format_='json')
        song.save_lyrics(filename=f"{LYRICS_DIR}/{song_name}_{artist_name}.txt",
                            overwrite=True, verbose=False, format_='txt')
        #TODO: Return lyrics
        
    else:
        print("Song not found.")
        return None

In [ ]:
scrape_lyrics("Eminem", "Mockingbird")